In [27]:
# Importing the libraries 
import pandas as pd
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [28]:
pip install --upgrade matplotlib

   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------------ --------------------- 3.7/8.0 MB 27.3 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 29.3 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.9.2
    Uninstalling matplotlib-3.9.2:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\63422\\anaconda3\\lib\\site-packages\\matplotlib\\backends\\_backend_agg.cp312-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import *
from torch.autograd import Variable
from torchvision import transforms
import torchvision
from tqdm import tqdm
from torchvision.utils import save_image
to_pil_image = transforms.ToPILImage()
def image_to_vid(images):
    imgs = [np.array(to_pil_image(img)) for img in images]
    imageio.mimsave('../outputs/generated_images.gif', imgs)
def save_reconstructed_images(recon_images, epoch):
    save_image(recon_images.cpu(), f"{epoch}.jpg")
def save_loss_plot(train_loss, valid_loss):
    # loss plots
    plt.figure(figsize=(10, 7))
    plt.plot(train_loss, color='orange', label='train loss')
    plt.plot(valid_loss, color='red', label='validataion loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('loss.jpg')
    plt.show()
from torchvision.utils import make_grid

In [4]:
class Encodercnn(nn.Module):
    def __init__(self,input_dim=2,out_dim=2,latent_dim=1):
        super(Encodercnn,self).__init__()

        self.enc = nn.Sequential(
            nn.Linear(input_dim,2*input_dim),
            nn.Tanh(),

            nn.Linear(2*input_dim,4*input_dim),
            nn.Tanh(),
            

            nn.Flatten(),
            nn.Linear(input_dim*4, 1*latent_dim),            

            
        )
    def forward(self, x):
        return self.enc(x)

In [5]:
class Decodercnn(nn.Module):
    def __init__(self, input_dim=2,out_dim=2,latent_dim=1):
        super(Decodercnn,self).__init__()
        

        self.dec = nn.Sequential(
            nn.Linear(latent_dim, input_dim*2),  
            nn.Tanh(),
            nn.Linear(input_dim*2,4*input_dim),
            nn.Tanh(),

            nn.Linear(input_dim*4, out_dim),            

            
        )
    def forward(self, x):
        
        
        
        return self.dec(x)


In [7]:
class DKGM(nn.Module):
    def __init__(self, N,device,input_dim=2,out_dim=2,latent_dim=1,m=100,T=20,a=0):
        super(DKGM, self).__init__()
        
        self.latent_dim = latent_dim
        #initial CNN
        self.encoder_initial=Encodercnn(input_dim=input_dim,out_dim=out_dim,latent_dim=latent_dim)
        self.decoder_initial=Decodercnn(input_dim=input_dim,out_dim=out_dim,latent_dim=latent_dim)
        self.bm=np.power(m,-2/9)
        self.T=T
        self.N=N
        self.a=a
        self.device=device
        

        #list of bias encoders
        self.bias_encoders=nn.ModuleList()
        for i in range(self.T):
            self.bias_encoders.append(Encodercnn(input_dim=input_dim,out_dim=out_dim,latent_dim=latent_dim))
        #list of bias decoders
        self.bias_decoders=nn.ModuleList()
        for i in range(self.T):
            self.bias_decoders.append(Decodercnn(input_dim=input_dim,out_dim=out_dim,latent_dim=latent_dim))
  
    def reparametrize(self,mu,log_r,log_beta):
        
        coef= self.bm #b(m)
        n=log_r.size(dim=0)
        d=log_r.size(dim=1)
        beta=torch.exp(0.5*log_beta)
        uniform_k=torch.rand(n*d,3, device=self.device)
        U_k=2*uniform_k-1
        
        uniform_p=torch.rand(n,d, device=self.device)
        U_p=2*uniform_p-1  #uniform prior
        
        median=torch.median(U_k,1)[0].reshape(n,d) #sample from E kernel
        sample=coef*(torch.exp(0.5*log_r)*median+mu)+beta*U_p
        return sample
       
    

    def forward(self,x):
        
        x=self.a*x+2
        #initial encoding
        enc=self.encoder_initial(x)


        #initial decoding
        reconstruction=self.decoder_initial(enc)
        
        #initialization of sequence EVAE
        total_recons=reconstruction
        bias=x-reconstruction
        recons_bias=torch.zeros_like(x, device=self.device)
        a_i=0
        #sequence of bias encoding +decoding 
        for i in range(self.T):
            bias_encoder=self.bias_encoders[i]
            bias_decoder=self.bias_decoders[i]
            
            bias=bias-recons_bias*a_i  
            z_b= bias_encoder(bias)

            recons_bias=bias_decoder(z_b)   
            a_i=1/(i+1.0)
            total_recons+=recons_bias*a_i


        return total_recons

In [8]:
def final_loss(mse_loss):
    #para1[0]: log_r  para1[1]:log_beta

    return mse_loss


In [9]:
def model_train(model,dataloader,dataset,device,optimizer,criterion):


    model.train()
    running_loss=0.0
    counter=0
    for i, data in tqdm(enumerate(dataloader),total=int(len(dataset)/dataloader.batch_size)):
        counter+=1
        #data=data[0]
        data=data.to(device)
        
        optimizer.zero_grad()
        reconstruction=model(data)

        mse_loss= criterion(reconstruction,data)

        loss=final_loss(mse_loss)
        #print(loss)

        loss.backward()
        running_loss+=loss.item()
        optimizer.step()

    train_loss=running_loss/counter
    #print(log_var)
    return train_loss
  

def model_validate(model,dataloader,dataset,device,criterion,optimizer):

    model.eval()
    running_loss=0.0
    counter=0
    with torch.no_grad():
        for i,data in tqdm(enumerate(dataloader),total=int(len(dataset)/dataloader.batch_size)):
            counter+=1
            #data=data[0]
            data=data.to(device)
            optimizer.zero_grad()
            reconstruction=model(data)
            mse_loss= criterion(reconstruction,data)

            loss=mse_loss#+bce_loss_bias#final_loss(bce_loss,mu,log_var)
            running_loss+=loss.item()
            if i==int(len(dataset)/dataloader.batch_size)-1:
                recon_images=reconstruction
        valid_loss=running_loss/counter
        return valid_loss,recon_images


# training DKGM

In [ ]:
import random

np.random.seed(42)
#Parameters to set
lambd=1
n=20000
alpha=4*np.pi/3
u=np.random.exponential(lambd, n) 

phi=np.random.normal(np.pi/2, np.sqrt(np.pi/4),n)

X=np.zeros((n, 2))
for i in range(n):
    
    x = alpha*np.sqrt(u[i])/3 *np.cos(alpha*np.sqrt(u[i]))
    y = alpha*np.sqrt(u[i])/3 *np.sin(alpha*np.sqrt(u[i]))
    X[i,0]=x
    X[i,1]=y

plt.scatter(X[:,0], X[:,1], c ="blue")

# # To show the plot
plt.show()

X=Variable(torch.from_numpy(X).float())

In [ ]:

ax=plt.gca()
sns.scatterplot(x=X[:,0],y=X[:,1],ax=ax)
plt.show()

In [21]:
X.size()

torch.Size([20000, 2])

In [13]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')


lr=0.0005
epochs=100
batch_size=100

model_kernel=DKGM(latent_dim=1,N=batch_size,T=30,device=device,a=0.1).to(device)
model_boost=DKGM(latent_dim=1,N=batch_size,T=30,device=device,a=0).to(device)
train_loader=torch.utils.data.DataLoader(X,batch_size=batch_size,shuffle=True,pin_memory=True)

optimizer_kernel=optim.Adam(model_kernel.parameters(),lr=lr)
optimizer_boost=optim.Adam(model_boost.parameters(),lr=lr)
#scheduler=optim.lr_scheduler.StepLR(optimizer,epochs//4,gamma=0.3)
criterion=torch.nn.MSELoss(reduction="sum")



In [22]:
grid_images=[]
train_loss=[]
valid_loss=[]
mus=[]
varss=[]


for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_kernel,train_loader,X,device,optimizer_kernel,criterion)

    train_loss.append(train_epoch_loss)


    print(f"train loss:{train_epoch_loss:.4f}")


Epoch1 of 100


100%|██████████| 200/200 [00:06<00:00, 30.24it/s]


train loss:184.5380
Epoch2 of 100


100%|██████████| 200/200 [00:06<00:00, 29.80it/s]


train loss:157.3244
Epoch3 of 100


...

train loss:0.0872
Epoch31 of 100


100%|██████████| 200/200 [00:06<00:00, 29.77it/s]


train loss:0.0837
Epoch32 of 100


  8%|▊         | 16/200 [00:00<00:06, 28.83it/s]


KeyboardInterrupt: 

In [23]:
reconstruct_X=model_kernel(X.to(device))
biased_image=0.1*X.to(device)+1

In [26]:

ax=plt.gca()
sns.scatterplot(x=reconstruct_X.cpu().detach().numpy()[:,0],y=reconstruct_X.cpu().detach().numpy()[:,1],ax=ax)

<Axes: xlabel='None', ylabel='None'>

In [25]:
ax=plt.gca()
sns.scatterplot(x=biased_image.cpu().detach().numpy()[:,0],y=biased_image.cpu().detach().numpy()[:,1],ax=ax)

<Axes: xlabel='None', ylabel='None'>

In [ ]:

lr=0.0005
epochs=100
batch_size=100

model_0=DKGM(latent_dim=1,N=batch_size,T=0,device=device,a=0.1).to(device)



optimizer0=optim.Adam(model_0.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_0,train_loader,X,device,optimizer0,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)
    #valid_loss.append(valid_epoch_loss)
  #  mus.append(mu)
   # varss.append(logvar.exp())
    #save images recon
    #save_reconstructed_images(recon_images,epoch+1)


    #grid_images.append(image_grid)
    print(f"train loss:{train_epoch_loss:.4f}")
    #print(f"valid loss:{valid_epoch_loss:.4f}")

In [ ]:
reconstruct_X0=model_0(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X0.cpu().detach().numpy()[:,0],y=reconstruct_X0.cpu().detach().numpy()[:,1],ax=ax)

In [ ]:

lr=0.0005
epochs=100
batch_size=100

model_1=DKGM(latent_dim=1,N=batch_size,T=1,device=device,a=0.1).to(device)



optimizer1=optim.Adam(model_1.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_1,train_loader,X,device,optimizer1,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)

    print(f"train loss:{train_epoch_loss:.4f}")
    #print(f"valid loss:{valid_epoch_loss:.4f}")

In [ ]:
reconstruct_X1=model_1(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X1.cpu().detach().numpy()[:,0],y=reconstruct_X1.cpu().detach().numpy()[:,1],ax=ax)

In [ ]:

lr=0.0001
epochs=50
batch_size=100

model_2=DKGM(latent_dim=1,N=batch_size,T=2,device=device,a=0.1).to(device)



optimizer2=optim.Adam(model_2.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_2,train_loader,X,device,optimizer2,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)

    print(f"train loss:{train_epoch_loss:.4f}")



In [ ]:
    
reconstruct_X2=model_2(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X2.cpu().detach().numpy()[:,0],y=reconstruct_X2.cpu().detach().numpy()[:,1],ax=ax)

In [ ]:

lr=0.0001
epochs=50
batch_size=100

model_3=DKGM(latent_dim=1,N=batch_size,T=3,device=device,a=0.1).to(device)



optimizer3=optim.Adam(model_3.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_3,train_loader,X,device,optimizer3,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)

    print(f"train loss:{train_epoch_loss:.4f}")

    
reconstruct_X3=model_3(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X3.cpu().detach().numpy()[:,0],y=reconstruct_X3.cpu().detach().numpy()[:,1],ax=ax)

In [ ]:

lr=0.0001
epochs=50
batch_size=100

model_4=DKGM(latent_dim=1,N=batch_size,T=4,device=device,a=0.1).to(device)



optimizer4=optim.Adam(model_4.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_4,train_loader,X,device,optimizer4,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)

    print(f"train loss:{train_epoch_loss:.4f}")

    
reconstruct_X4=model_4(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X4.cpu().detach().numpy()[:,0],y=reconstruct_X4.cpu().detach().numpy()[:,1],ax=ax)

In [ ]:

lr=0.0005
epochs=100
batch_size=100

model_5=DKGM(latent_dim=1,N=batch_size,T=5,device=device,a=0.1).to(device)



optimizer5=optim.Adam(model_5.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_5,train_loader,X,device,optimizer5,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)

    print(f"train loss:{train_epoch_loss:.4f}")

    
reconstruct_X5=model_5(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X5.cpu().detach().numpy()[:,0],y=reconstruct_X5.cpu().detach().numpy()[:,1],ax=ax)

In [ ]:

lr=0.0005
epochs=100
batch_size=100

model_10=DKGM(latent_dim=1,N=batch_size,T=10,device=device,a=0.1).to(device)



optimizer10=optim.Adam(model_10.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_10,train_loader,X,device,optimizer10,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)

    print(f"train loss:{train_epoch_loss:.4f}")

    
reconstruct_X10=model_10(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X10.cpu().detach().numpy()[:,0],y=reconstruct_X10.cpu().detach().numpy()[:,1],ax=ax)

In [ ]:

lr=0.0005
epochs=50
batch_size=100

model_15=DKGM(latent_dim=1,N=batch_size,T=15,device=device,a=0.1).to(device)



optimizer15=optim.Adam(model_15.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_15,train_loader,X,device,optimizer15,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)

    print(f"train loss:{train_epoch_loss:.4f}")

    
reconstruct_X15=model_15(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X15.cpu().detach().numpy()[:,0],y=reconstruct_X15.cpu().detach().numpy()[:,1],ax=ax)

In [ ]:

lr=0.0005
epochs=50
batch_size=100

model_20=DKGM(latent_dim=1,N=batch_size,T=20,device=device,a=0.1).to(device)



optimizer20=optim.Adam(model_20.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_20,train_loader,X,device,optimizer20,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)

    print(f"train loss:{train_epoch_loss:.4f}")

    
reconstruct_X20=model_20(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X20.cpu().detach().numpy()[:,0],y=reconstruct_X20.cpu().detach().numpy()[:,1],ax=ax)

In [1]:
lr=0.0005
epochs=50
batch_size=100

model_25=DKGM(latent_dim=1,N=batch_size,T=25,device=device,a=0.1).to(device)



optimizer25=optim.Adam(model_25.parameters(),lr=lr)



for epoch in range(epochs):
    print(f"Epoch{epoch+1} of {epochs}")
    train_epoch_loss=model_train(model_25,train_loader,X,device,optimizer25,criterion)
    #valid_epoch_loss,recon_images=model_validate(model,test_loader,test_set,device,criterion)
    train_loss.append(train_epoch_loss)

    print(f"train loss:{train_epoch_loss:.4f}")

    
reconstruct_X25=model_25(X.to(device))
ax=plt.gca()
sns.scatterplot(x=reconstruct_X25.cpu().detach().numpy()[:,0],y=reconstruct_X25.cpu().detach().numpy()[:,1],ax=ax)

NameError: name 'DKGM' is not defined